# Exponential smoothing

statsmodels.tsa.holtwinters.ExponentialSmoothing [link][1]

[1]: https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html

Модель может быть аддитивная или мультипликативная. У нас есть:
- Level
- Trend
- Seasonality
- Noise


**Исторически Экспоненциальное сглаживание развивалось в 3 шага **

## MODEL 1. 1940-50 Brown's simple exponential smoothing
Нет тренда, нет сезонной составляющей, 
$ x_t $ - исторические значения ряда, $ s_t $ - модельные значения (продгонка = угадывание закономерности, которую мы используем для будущих `t`), $ 0 < \alpha < 1$.

**Модель**

Представление модели: *"подруливание"*

$$ s_0 = x_0 $$
$$ s_t = s_{t-1} + \alpha \cdot (x_{t-1} - s_{t-1})$$


Представление модели: *взвешенное среднее*

$$ s_0 = x_0 $$
$$ s_t = \alpha \cdot x_{t-1} + ( 1 - \alpha ) \cdot s_{t-1}$$

Браун решал задачу прогнозирования складских запасов. (на 4 - 7 наблюдений вперед, на 12 - уже сомнительно)

Метод хороший для краткосрочного планирования. 

Если раскрыть уравнение, то получим:

$$ s_t = \alpha \cdot ( x_{t-1} + ( 1 - \alpha ) \cdot x_{t-2} + ( 1 - \alpha )^2 \cdot x_{t-3} + ( 1 - \alpha )^3 \cdot x_{t-4} + ... ) $$

Таким образом,  модельное значение ($s_t$) - это подгонка/линейная комбинация всех предыдущих значений. Отметим, чем более раннее значение берется, тем меньше его вес. 
Хорошо ложиться в парадигму KISS (simple methods survive).

Множители при $x$ убывают в геометрической прогрессии (экспоненциально) ($ 0 < \alpha < 1$).

Почему сглаживание? - мы берем сумму всех предыдущих значений ряда, а сумма коэффициентов перед $x_{t-...}$ равна 1.

То есть у нас такое сглаживание, что веса при $x_{t-...}$ убывают экспоненциально.

## MODEL 2. Double exponential smoothing, Holt-Winters double exponential smoothing

Есть тренд, нет сезонной составляющей
Тренд линейный, локальный.

Локальный тренд = тренд разный для каждого момента времени, постоянно меняющийся, постоянно модифицирующийся от наблюдений к наблюдению. (похоже на кусочно-линейный, но в кусочно-линейном нет постоянной смены)

Замечание. Возможно моделирование нелинейным трендом. Выигрыша в качестве модели нет. 
Усложнения в виде выбора более сложного/хитрого тренда - не приносят выигрыша в качестве.

$s_t$ - "сглаженное" значение временного ряда (модельное, теоретическое значение)
$b_t$ - локальное значение тренда

$$ s_1 = x_1 $$
$$ b_1 = x_1 - x_0 $$

Уравнение для прогнозирования
$$ \hat{y}_{t+h} = s_t + h \cdot b_t $$

Уравнение для уровня ряда
$$ s_t = \alpha \cdot x_{t-1} + ( 1 - \alpha ) \cdot (s_{t-1} + b_{t-1})$$

Уравнение для тренда
$$ b_t = \beta \cdot (s_t - s_{t-1}) + ( 1 - \beta ) \cdot b_{t-1}$$


### Линейный тренд для временных рядов. Модифицированные уравнения

Уравнение для прямой линии:
$$ y = a + b \cdot x $$
Если мы отслеживаем изменения в дискретные моменты времени, то:
$$ y_t = a + b \cdot t $$

t - номер наблюдения

$$ y_t - y_{t-1} = (a + b \cdot t ) - (a + b \cdot (t-1))$$ 
$$ y_t - y_{t-1} = b$$ 

## MODEL 3. Triple exponential smoothing, Holt-Winters additive method

Есть тренд, есть сезонная составляющая.
Тренд линейный, локальный.
Сезонность локальная.

T - период сезонных отклонений
S_t - локальные сезонные поправки.

Уравнение для прогнозирования
$$ \hat{y}_{t+m} = L_{t} + m \cdot b_t + S_{t+m-T}$$


Уравнение для уровня ряда
$$ s_t = \alpha \cdot ( x_{t-1} - S_{t-T}) + ( 1 - \alpha ) \cdot (s_{t-1} + b_{t-1})$$

Уравнение для тренда
$$ b_t = \beta \cdot (s_{t} - s_{t-1}) + ( 1 - \beta ) \cdot b_{t-1} $$

Уравнение для сезонной составляющей
$$ S_t = \gamma \cdot (x_{t} - s_{t-1} - b_{t-1}) + ( 1 - \gamma ) \cdot S_{t-T}$$


$ S_{t-T} $ - значение сезонности в предыдущий сезон

$x_{t} - S_{t-T}$ - значение ряда очищенное от сезонности

$\alpha, \beta, \gamma $ - коэффициенты подруливания

Как их находят, в процессе перебора разумных значений:
- выбрали значения,
- построили модель, 
- посчитали MAPE (MAE, MSE) для выбранного набора
